In [1]:
import pandas as pd
import sys
sys.path.append('/Users/halukamb/Documents/CodeRelated/HBDataviz_new/2025/01_BTW100Deutsche/Wrangle/lib/python3.12/site-packages')
import numpy as np
from json import loads, dumps

In [2]:
! pip install pyreadstat


[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip


In [152]:
def germanElectorate(df):
    german_df=df[df["german"]=="JA"]
    german_df=german_df.dropna(subset="age")
    german_df["age"]=german_df["age"].astype(float)
    german_electorate=german_df[german_df["age"]>=18]
    return german_electorate

In [172]:
def reformgroups(df):
    df["Alter"]=None
    df["Alter"]=np.where(df["age"]<30,"U30",df["Alter"])
    df["Alter"]=np.where(df["age"]>=30,"30-49",df["Alter"])
    df["Alter"]=np.where(df["age"]>=50,"50-64",df["Alter"])
    df["Alter"]=np.where(df["age"]>64,"Ü65",df["Alter"])
    df["Geschlecht"]=np.nan
    df["Geschlecht"]=np.where(df["sex"]=="FRAU","Weiblich",df["Geschlecht"])
    df["Geschlecht"]=np.where(df["sex"]=="MANN","Männlich",df["Geschlecht"])
    df["OstWest"]=np.nan
    df["OstWest"]=np.where(df["eastwest"]=="ALTE BUNDESLAENDER","West",df["OstWest"])
    df["OstWest"]=np.where(df["eastwest"]=="NEUE BUNDESLAENDER","Ost",df["OstWest"])
    df["OstWest"]=np.where(df["land"].str.contains("BERLIN"),"Berlin",df["OstWest"])
    df["Besiedelung"]=np.nan
    df["Besiedelung"]=np.where(df["gs01"].isin(['GROSSSTADT']),"dicht besiedelt",df["Besiedelung"])
    df["Besiedelung"]=np.where(df["gs01"].isin( ['MITTEL-, KLEINSTADT', 'VORORT GROSSSTADT']),"mittlere Besiedlungsdichte",df["Besiedelung"])
    df["Besiedelung"]=np.where(df["gs01"].isin( ['LAENDL. DORF','EINZELHAUS, LAND']),"gering besiedelt",df["Besiedelung"])
    
    


    
    return df

In [173]:
df_2018 = pd.read_spss("../data/AllbusData/ZA5270_v2-0-0.sav")
df_2018 = germanElectorate(df_2018)
df_2018= reformgroups(df_2018)
df_2018.Alter.unique()

array(['50-64', 'U30', '30-49', 'Ü65'], dtype=object)

In [174]:
df_2021 = pd.read_spss("../data/AllbusData/ZA5280_v2-0-1.sav")
df_2021 = germanElectorate(df_2021)
df_2021= reformgroups(df_2021)

In [175]:
df_2023 = pd.read_spss("../data/AllbusData/ZA8830_v1-0-0.sav")
df_2023 = germanElectorate(df_2023)
df_2023= reformgroups(df_2023)


In [176]:
for i in df_2023["land"].unique():
    print(i)

RHEINLAND-PFALZ
BAYERN
THUERINGEN
SCHLESWIG-HOLSTEIN
BADEN-WUERTTEMBERG
NIEDERSACHSEN
BRANDENBURG
NORDRHEIN-WESTFALEN
SACHSEN
HESSEN
EHEM. BERLIN-OST
EHEM. BERLIN-WEST
MECKLENB.-VORPOMMERN
SACHSEN-ANHALT
HAMBURG
SAARLAND
BREMEN


In [177]:
factor_list=["Alter","Geschlecht","OstWest","Besiedelung"]

In [178]:
def analyze_factor_patterns(dataset,variable_to_check):
    base_df=pd.DataFrame()
    for element in factor_list:
        #print(element)
        for element2 in factor_list:
            #print(element2)
            baselist=[variable_to_check,"za_nr"]
            extralist=[element,element2]
            extralist=list(set(extralist))
    
            baselist.extend(extralist)
            dataset_reduced=dataset[baselist]
            dataset_reduced = dataset_reduced[~dataset_reduced.isin(["nan"]).any(axis=1)]
            if len(dataset_reduced)<50:
                print("low")
            extralist_plus_factor=extralist+[variable_to_check]
            grouped_df=dataset_reduced.groupby(by=extralist_plus_factor).count()
            grouped_df=grouped_df.dropna(axis=1)
            grouped_df['percent'] = grouped_df['za_nr']*100 / grouped_df.groupby(by=extralist)['za_nr'].transform('sum')
            grouped_df['absolute'] = grouped_df['za_nr']
            
            grouped_df=grouped_df.reset_index()
            base_df=pd.concat([base_df,grouped_df],axis=0)
    base_df=base_df.drop_duplicates()
    base_df_cols=['Alter', 'Besiedelung', 'Geschlecht', 'OstWest', variable_to_check,
       'absolute','percent']
    
    return base_df[base_df_cols]

In [179]:
df_2023_inflation=analyze_factor_patterns(df_2023,"va03")
df_2023_inflation

/var/folders/jf/2jm804y50jv2nrhclzdyvvdh0000gn/T/ipykernel_11116/2742315680.py:17: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_df=dataset_reduced.groupby(by=extralist_plus_factor).count()
/var/folders/jf/2jm804y50jv2nrhclzdyvvdh0000gn/T/ipykernel_11116/2742315680.py:17: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_df=dataset_reduced.groupby(by=extralist_plus_factor).count()
/var/folders/jf/2jm804y50jv2nrhclzdyvvdh0000gn/T/ipykernel_11116/2742315680.py:17: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False

,Alter,Besiedelung,Geschlecht,OstWest,va03,absolute,percent
0,30-49,NaN,NaN,NaN,AM DRITTWICHTIGSTEN,342,26.677067
1,30-49,NaN,NaN,NaN,AM VIERTWICHTIGSTEN,425,33.151326
2,30-49,NaN,NaN,NaN,AM WICHTIGSTEN,213,16.614665
3,30-49,NaN,NaN,NaN,AM ZWEITWICHTIGSTEN,302,23.556942
4,50-64,NaN,NaN,NaN,AM DRITTWICHTIGSTEN,396,28.989751
...,...,...,...,...,...,...,...
7,NaN,gering besiedelt,NaN,NaN,AM ZWEITWICHTIGSTEN,292,21.581670
8,NaN,mittlere Besiedlungsdichte,NaN,NaN,AM DRITTWICHTIGSTEN,613,27.366071
9,NaN,mittlere Besiedlungsdichte,NaN,NaN,AM VIERTWICHTIGSTEN,895,39.955357
10,NaN,mittlere Besiedlungsdichte,NaN,NaN,AM WICHTIGSTEN,260,11.607143


In [180]:
df_2021_inflation=analyze_factor_patterns(df_2021,"va03")
df_2021_inflation

/var/folders/jf/2jm804y50jv2nrhclzdyvvdh0000gn/T/ipykernel_11116/2742315680.py:17: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_df=dataset_reduced.groupby(by=extralist_plus_factor).count()
/var/folders/jf/2jm804y50jv2nrhclzdyvvdh0000gn/T/ipykernel_11116/2742315680.py:17: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_df=dataset_reduced.groupby(by=extralist_plus_factor).count()
/var/folders/jf/2jm804y50jv2nrhclzdyvvdh0000gn/T/ipykernel_11116/2742315680.py:17: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False

,Alter,Besiedelung,Geschlecht,OstWest,va03,absolute,percent
0,30-49,NaN,NaN,NaN,AM DRITTWICHTIGSTEN,223,26.143025
1,30-49,NaN,NaN,NaN,AM VIERTWICHTIGSTEN,405,47.479484
2,30-49,NaN,NaN,NaN,AM WICHTIGSTEN,74,8.675264
3,30-49,NaN,NaN,NaN,AM ZWEITWICHTIGSTEN,151,17.702227
4,50-64,NaN,NaN,NaN,AM DRITTWICHTIGSTEN,242,24.007937
...,...,...,...,...,...,...,...
7,NaN,gering besiedelt,NaN,NaN,AM ZWEITWICHTIGSTEN,163,16.979167
8,NaN,mittlere Besiedlungsdichte,NaN,NaN,AM DRITTWICHTIGSTEN,376,25.066667
9,NaN,mittlere Besiedlungsdichte,NaN,NaN,AM VIERTWICHTIGSTEN,837,55.800000
10,NaN,mittlere Besiedlungsdichte,NaN,NaN,AM WICHTIGSTEN,66,4.400000


In [181]:
comp_inflation=pd.merge(df_2021_inflation,df_2023_inflation,on=['Alter', 'Besiedelung', 'Geschlecht', 'OstWest', 'va03'],suffixes=["_2021","_2023"])
comp_inflation

,Alter,Besiedelung,Geschlecht,OstWest,va03,absolute_2021,percent_2021,absolute_2023,percent_2023
0,30-49,NaN,NaN,NaN,AM DRITTWICHTIGSTEN,223,26.143025,342,26.677067
1,30-49,NaN,NaN,NaN,AM VIERTWICHTIGSTEN,405,47.479484,425,33.151326
2,30-49,NaN,NaN,NaN,AM WICHTIGSTEN,74,8.675264,213,16.614665
3,30-49,NaN,NaN,NaN,AM ZWEITWICHTIGSTEN,151,17.702227,302,23.556942
4,50-64,NaN,NaN,NaN,AM DRITTWICHTIGSTEN,242,24.007937,396,28.989751
...,...,...,...,...,...,...,...,...,...
255,NaN,gering besiedelt,NaN,NaN,AM ZWEITWICHTIGSTEN,163,16.979167,292,21.581670
256,NaN,mittlere Besiedlungsdichte,NaN,NaN,AM DRITTWICHTIGSTEN,376,25.066667,613,27.366071
257,NaN,mittlere Besiedlungsdichte,NaN,NaN,AM VIERTWICHTIGSTEN,837,55.800000,895,39.955357
258,NaN,mittlere Besiedlungsdichte,NaN,NaN,AM WICHTIGSTEN,66,4.400000,260,11.607143


In [182]:
df_2021_party=analyze_factor_patterns(df_2021,"pv01")
df_2021_party

/var/folders/jf/2jm804y50jv2nrhclzdyvvdh0000gn/T/ipykernel_11116/2742315680.py:17: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_df=dataset_reduced.groupby(by=extralist_plus_factor).count()
/var/folders/jf/2jm804y50jv2nrhclzdyvvdh0000gn/T/ipykernel_11116/2742315680.py:17: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_df=dataset_reduced.groupby(by=extralist_plus_factor).count()
/var/folders/jf/2jm804y50jv2nrhclzdyvvdh0000gn/T/ipykernel_11116/2742315680.py:17: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False

,Alter,Besiedelung,Geschlecht,OstWest,pv01,absolute,percent
0,30-49,NaN,NaN,NaN,AFD,75,7.324219
1,30-49,NaN,NaN,NaN,ANDERE PARTEI,62,6.054688
2,30-49,NaN,NaN,NaN,CDU-CSU,215,20.996094
3,30-49,NaN,NaN,NaN,DIE GRUENEN,306,29.882812
4,30-49,NaN,NaN,NaN,DIE LINKE,69,6.738281
...,...,...,...,...,...,...,...
19,NaN,mittlere Besiedlungsdichte,NaN,NaN,DIE GRUENEN,432,23.052295
20,NaN,mittlere Besiedlungsdichte,NaN,NaN,DIE LINKE,145,7.737460
21,NaN,mittlere Besiedlungsdichte,NaN,NaN,FDP,230,12.273212
22,NaN,mittlere Besiedlungsdichte,NaN,NaN,SPD,296,15.795091


In [183]:
df_2023_party=analyze_factor_patterns(df_2023,"pv01")
df_2023_party

/var/folders/jf/2jm804y50jv2nrhclzdyvvdh0000gn/T/ipykernel_11116/2742315680.py:17: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_df=dataset_reduced.groupby(by=extralist_plus_factor).count()
/var/folders/jf/2jm804y50jv2nrhclzdyvvdh0000gn/T/ipykernel_11116/2742315680.py:17: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_df=dataset_reduced.groupby(by=extralist_plus_factor).count()
/var/folders/jf/2jm804y50jv2nrhclzdyvvdh0000gn/T/ipykernel_11116/2742315680.py:17: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False

,Alter,Besiedelung,Geschlecht,OstWest,pv01,absolute,percent
0,30-49,NaN,NaN,NaN,AFD,160,15.717092
1,30-49,NaN,NaN,NaN,ANDERE PARTEI,47,4.616896
2,30-49,NaN,NaN,NaN,CDU-CSU,206,20.235756
3,30-49,NaN,NaN,NaN,DIE GRUENEN,214,21.021611
4,30-49,NaN,NaN,NaN,DIE LINKE,67,6.581532
...,...,...,...,...,...,...,...
19,NaN,mittlere Besiedlungsdichte,NaN,NaN,DIE GRUENEN,322,17.349138
20,NaN,mittlere Besiedlungsdichte,NaN,NaN,DIE LINKE,115,6.196121
21,NaN,mittlere Besiedlungsdichte,NaN,NaN,FDP,162,8.728448
22,NaN,mittlere Besiedlungsdichte,NaN,NaN,SPD,371,19.989224


In [184]:
comp_inflation["diff_percent"]=comp_inflation["percent_2023"]-comp_inflation["percent_2021"]
comp_inflation[comp_inflation["va03"]=="AM WICHTIGSTEN"].sort_values(by="diff_percent",ascending=False)
#comp_inflation

,Alter,Besiedelung,Geschlecht,OstWest,va03,absolute_2021,percent_2021,absolute_2023,percent_2023,diff_percent
106,U30,dicht besiedelt,NaN,NaN,AM WICHTIGSTEN,7,6.481481,43,22.395833,15.914352
90,U30,NaN,NaN,West,AM WICHTIGSTEN,18,6.870229,89,21.394231,14.524002
26,U30,NaN,Männlich,NaN,AM WICHTIGSTEN,13,7.428571,58,19.931271,12.502700
138,U30,mittlere Besiedlungsdichte,NaN,NaN,AM WICHTIGSTEN,11,8.208955,50,20.491803,12.282848
42,U30,NaN,Weiblich,NaN,AM WICHTIGSTEN,19,10.674157,64,22.939068,12.264911
...,...,...,...,...,...,...,...,...,...,...
70,50-64,NaN,NaN,Ost,AM WICHTIGSTEN,13,4.100946,26,6.235012,2.134066
118,50-64,gering besiedelt,NaN,NaN,AM WICHTIGSTEN,15,4.249292,29,6.209850,1.960558
54,50-64,NaN,NaN,Berlin,AM WICHTIGSTEN,2,4.081633,3,5.357143,1.275510
110,Ü65,dicht besiedelt,NaN,NaN,AM WICHTIGSTEN,6,4.054054,11,4.471545,0.417491


In [185]:
comp_party=pd.merge(df_2021_party,df_2023_party,on=['Alter', 'Besiedelung', 'Geschlecht', 'OstWest', 'pv01'],suffixes=["_2021","_2023"])
comp_party["diff_percent"]=comp_party["percent_2023"]-comp_party["percent_2021"]
comp_party.sort_values("diff_percent")
comp_party_pivot=comp_party.pivot(index=['Alter','Besiedelung','Geschlecht','OstWest'], columns='pv01')[['percent_2021','percent_2023',"absolute_2021","absolute_2023"]].reset_index()

new_columns=[]
for index in comp_party_pivot:
    if index[1]!="":
        new_columns.append(index[1]+"_"+index[0])
    else:
        new_columns.append(index[0])
new_columns
comp_party_pivot.columns=new_columns
comp_party_pivot

,Alter,Besiedelung,Geschlecht,OstWest,AFD_percent_2021,ANDERE PARTEI_percent_2021,CDU-CSU_percent_2021,DIE GRUENEN_percent_2021,DIE LINKE_percent_2021,FDP_percent_2021,...,SPD_absolute_2021,WUERDE NICHT WAEHLEN_absolute_2021,AFD_absolute_2023,ANDERE PARTEI_absolute_2023,CDU-CSU_absolute_2023,DIE GRUENEN_absolute_2023,DIE LINKE_absolute_2023,FDP_absolute_2023,SPD_absolute_2023,WUERDE NICHT WAEHLEN_absolute_2023
0,NaN,NaN,NaN,Berlin,5.681818,3.977273,19.318182,28.409091,9.659091,5.113636,...,36,13,19,8,30,45,39,9,35,9
1,NaN,NaN,NaN,Ost,11.690496,5.298570,23.801514,14.297729,13.204373,11.522288,...,162,78,235,48,232,117,129,66,215,68
2,NaN,NaN,NaN,West,4.579208,3.547855,27.681518,27.310231,4.867987,12.458746,...,369,105,201,99,670,546,76,230,488,120
3,NaN,NaN,Männlich,NaN,9.308240,3.916582,25.584944,20.345880,7.985758,13.123093,...,295,93,278,81,488,318,111,177,360,101
4,NaN,NaN,Männlich,Berlin,8.791209,3.296703,15.384615,26.373626,8.791209,7.692308,...,20,7,10,5,15,19,17,4,16,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,Ü65,NaN,Männlich,NaN,8.452951,1.275917,31.897927,13.237640,7.814992,11.642743,...,136,25,60,7,205,62,36,41,162,25
61,Ü65,NaN,Weiblich,NaN,2.226345,2.226345,36.920223,15.213358,8.534323,10.018553,...,109,25,37,14,174,82,31,35,164,27
62,Ü65,dicht besiedelt,NaN,NaN,6.741573,1.685393,30.898876,15.168539,5.617978,8.426966,...,46,10,16,4,50,23,17,17,78,9
63,Ü65,gering besiedelt,NaN,NaN,6.329114,1.898734,33.544304,13.291139,7.911392,10.759494,...,64,19,24,8,124,41,10,19,77,12


In [186]:
comp_inflation_pivot=comp_inflation.pivot(index=['Alter','Besiedelung','Geschlecht','OstWest'], columns='va03')[['percent_2021','percent_2023',"absolute_2021","absolute_2023"]].reset_index()
new_columns=[]
for index in comp_inflation_pivot:
    if index[1]!="":
        new_columns.append(index[1]+"_"+index[0])
    else:
        new_columns.append(index[0])
new_columns
comp_inflation_pivot.columns=new_columns
comp_inflation_pivot

,Alter,Besiedelung,Geschlecht,OstWest,AM DRITTWICHTIGSTEN_percent_2021,AM VIERTWICHTIGSTEN_percent_2021,AM WICHTIGSTEN_percent_2021,AM ZWEITWICHTIGSTEN_percent_2021,AM DRITTWICHTIGSTEN_percent_2023,AM VIERTWICHTIGSTEN_percent_2023,AM WICHTIGSTEN_percent_2023,AM ZWEITWICHTIGSTEN_percent_2023,AM DRITTWICHTIGSTEN_absolute_2021,AM VIERTWICHTIGSTEN_absolute_2021,AM WICHTIGSTEN_absolute_2021,AM ZWEITWICHTIGSTEN_absolute_2021,AM DRITTWICHTIGSTEN_absolute_2023,AM VIERTWICHTIGSTEN_absolute_2023,AM WICHTIGSTEN_absolute_2023,AM ZWEITWICHTIGSTEN_absolute_2023
0,NaN,NaN,NaN,Berlin,25.477707,47.770701,7.006369,19.745223,29.515419,35.682819,11.013216,23.788546,40,75,11,31,67,81,25,54
1,NaN,NaN,NaN,Ost,25.974026,51.048951,6.293706,16.683317,26.083815,40.895954,11.560694,21.459538,260,511,63,167,361,566,160,297
2,NaN,NaN,NaN,West,24.885496,56.284987,4.580153,14.249364,28.441907,39.355272,11.517797,20.685024,489,1106,90,280,847,1172,343,616
3,NaN,NaN,Männlich,NaN,24.984147,55.231452,5.009512,14.774889,29.456328,41.844920,9.848485,18.850267,394,871,79,233,661,939,221,423
4,NaN,NaN,Männlich,Berlin,19.718310,57.746479,5.633803,16.901408,33.962264,42.452830,8.490566,15.094340,14,41,4,12,36,45,9,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,Ü65,NaN,Männlich,NaN,24.481328,57.883817,2.489627,15.145228,28.251121,46.935725,5.829596,18.983558,118,279,12,73,189,314,39,127
61,Ü65,NaN,Weiblich,NaN,24.764151,57.311321,2.122642,15.801887,25.829726,44.877345,7.792208,21.500722,105,243,9,67,179,311,54,149
62,Ü65,dicht besiedelt,NaN,NaN,26.351351,52.702703,4.054054,16.891892,25.609756,50.813008,4.471545,19.105691,39,78,6,25,63,125,11,47
63,Ü65,gering besiedelt,NaN,NaN,24.166667,60.833333,1.666667,13.333333,26.984127,44.179894,7.936508,20.899471,58,146,4,32,102,167,30,79


In [187]:
comp_inflation_party_pivot=pd.merge(comp_inflation_pivot,comp_party_pivot,on=['Alter', 'Besiedelung', 'Geschlecht', 'OstWest'])
comp_inflation_party_pivot

,Alter,Besiedelung,Geschlecht,OstWest,AM DRITTWICHTIGSTEN_percent_2021,AM VIERTWICHTIGSTEN_percent_2021,AM WICHTIGSTEN_percent_2021,AM ZWEITWICHTIGSTEN_percent_2021,AM DRITTWICHTIGSTEN_percent_2023,AM VIERTWICHTIGSTEN_percent_2023,...,SPD_absolute_2021,WUERDE NICHT WAEHLEN_absolute_2021,AFD_absolute_2023,ANDERE PARTEI_absolute_2023,CDU-CSU_absolute_2023,DIE GRUENEN_absolute_2023,DIE LINKE_absolute_2023,FDP_absolute_2023,SPD_absolute_2023,WUERDE NICHT WAEHLEN_absolute_2023
0,NaN,NaN,NaN,Berlin,25.477707,47.770701,7.006369,19.745223,29.515419,35.682819,...,36,13,19,8,30,45,39,9,35,9
1,NaN,NaN,NaN,Ost,25.974026,51.048951,6.293706,16.683317,26.083815,40.895954,...,162,78,235,48,232,117,129,66,215,68
2,NaN,NaN,NaN,West,24.885496,56.284987,4.580153,14.249364,28.441907,39.355272,...,369,105,201,99,670,546,76,230,488,120
3,NaN,NaN,Männlich,NaN,24.984147,55.231452,5.009512,14.774889,29.456328,41.844920,...,295,93,278,81,488,318,111,177,360,101
4,NaN,NaN,Männlich,Berlin,19.718310,57.746479,5.633803,16.901408,33.962264,42.452830,...,20,7,10,5,15,19,17,4,16,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,Ü65,NaN,Männlich,NaN,24.481328,57.883817,2.489627,15.145228,28.251121,46.935725,...,136,25,60,7,205,62,36,41,162,25
61,Ü65,NaN,Weiblich,NaN,24.764151,57.311321,2.122642,15.801887,25.829726,44.877345,...,109,25,37,14,174,82,31,35,164,27
62,Ü65,dicht besiedelt,NaN,NaN,26.351351,52.702703,4.054054,16.891892,25.609756,50.813008,...,46,10,16,4,50,23,17,17,78,9
63,Ü65,gering besiedelt,NaN,NaN,24.166667,60.833333,1.666667,13.333333,26.984127,44.179894,...,64,19,24,8,124,41,10,19,77,12


In [188]:
comp_inflation_party_pivot.columns

Index(['Alter', 'Besiedelung', 'Geschlecht', 'OstWest',
       'AM DRITTWICHTIGSTEN_percent_2021', 'AM VIERTWICHTIGSTEN_percent_2021',
       'AM WICHTIGSTEN_percent_2021', 'AM ZWEITWICHTIGSTEN_percent_2021',
       'AM DRITTWICHTIGSTEN_percent_2023', 'AM VIERTWICHTIGSTEN_percent_2023',
       'AM WICHTIGSTEN_percent_2023', 'AM ZWEITWICHTIGSTEN_percent_2023',
       'AM DRITTWICHTIGSTEN_absolute_2021',
       'AM VIERTWICHTIGSTEN_absolute_2021', 'AM WICHTIGSTEN_absolute_2021',
       'AM ZWEITWICHTIGSTEN_absolute_2021',
       'AM DRITTWICHTIGSTEN_absolute_2023',
       'AM VIERTWICHTIGSTEN_absolute_2023', 'AM WICHTIGSTEN_absolute_2023',
       'AM ZWEITWICHTIGSTEN_absolute_2023', 'AFD_percent_2021',
       'ANDERE PARTEI_percent_2021', 'CDU-CSU_percent_2021',
       'DIE GRUENEN_percent_2021', 'DIE LINKE_percent_2021',
       'FDP_percent_2021', 'SPD_percent_2021',
       'WUERDE NICHT WAEHLEN_percent_2021', 'AFD_percent_2023',
       'ANDERE PARTEI_percent_2023', 'CDU-CSU_percent_202

In [189]:
zensus=pd.read_csv("json_none.csv")
zensus

,Geschlecht,Besiedelung,OstWest,Alter,Share
0,NaN,NaN,Berlin,U30,0.662670
1,NaN,NaN,Berlin,50-64,1.008255
2,NaN,NaN,Berlin,Ü65,1.054885
3,NaN,NaN,Berlin,30-49,1.264092
4,Männlich,NaN,Berlin,NaN,1.910200
...,...,...,...,...,...
58,Weiblich,NaN,West,NaN,41.055375
59,NaN,mittlere Besiedlungsdichte,NaN,NaN,44.163973
60,Männlich,NaN,NaN,NaN,48.323387
61,Weiblich,NaN,NaN,NaN,51.619587


In [191]:
comp_inflation_party_pivot_zensus=pd.merge(comp_inflation_party_pivot,zensus,on=['Alter', 'Besiedelung', 'Geschlecht', 'OstWest'])
comp_inflation_party_pivot_zensus

,Alter,Besiedelung,Geschlecht,OstWest,AM DRITTWICHTIGSTEN_percent_2021,AM VIERTWICHTIGSTEN_percent_2021,AM WICHTIGSTEN_percent_2021,AM ZWEITWICHTIGSTEN_percent_2021,AM DRITTWICHTIGSTEN_percent_2023,AM VIERTWICHTIGSTEN_percent_2023,...,WUERDE NICHT WAEHLEN_absolute_2021,AFD_absolute_2023,ANDERE PARTEI_absolute_2023,CDU-CSU_absolute_2023,DIE GRUENEN_absolute_2023,DIE LINKE_absolute_2023,FDP_absolute_2023,SPD_absolute_2023,WUERDE NICHT WAEHLEN_absolute_2023,Share
0,NaN,NaN,NaN,Berlin,25.477707,47.770701,7.006369,19.745223,29.515419,35.682819,...,13,19,8,30,45,39,9,35,9,3.989901
1,NaN,NaN,NaN,Ost,25.974026,51.048951,6.293706,16.683317,26.083815,40.895954,...,78,235,48,232,117,129,66,215,68,16.433332
2,NaN,NaN,NaN,West,24.885496,56.284987,4.580153,14.249364,28.441907,39.355272,...,105,201,99,670,546,76,230,488,120,79.519741
3,NaN,NaN,Männlich,NaN,24.984147,55.231452,5.009512,14.774889,29.456328,41.844920,...,93,278,81,488,318,111,177,360,101,48.323387
4,NaN,NaN,Männlich,Berlin,19.718310,57.746479,5.633803,16.901408,33.962264,42.452830,...,7,10,5,15,19,17,4,16,4,1.910200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,Ü65,NaN,Männlich,NaN,24.481328,57.883817,2.489627,15.145228,28.251121,46.935725,...,25,60,7,205,62,36,41,162,25,12.463294
59,Ü65,NaN,Weiblich,NaN,24.764151,57.311321,2.122642,15.801887,25.829726,44.877345,...,25,37,14,174,82,31,35,164,27,15.997430
60,Ü65,dicht besiedelt,NaN,NaN,26.351351,52.702703,4.054054,16.891892,25.609756,50.813008,...,10,16,4,50,23,17,17,78,9,9.247415
61,Ü65,gering besiedelt,NaN,NaN,24.166667,60.833333,1.666667,13.333333,26.984127,44.179894,...,19,24,8,124,41,10,19,77,12,6.025729


In [197]:
comp_inflation_party_pivot_zensus.Alter.unique()

array([nan, '30-49', '50-64', 'U30', '65plus'], dtype=object)

In [198]:
comp_inflation_party_pivot_zensus.Alter=comp_inflation_party_pivot_zensus.Alter.str.replace("Ü65","65plus")

In [199]:
comp_inflation_party_pivot_zensus.Besiedelung.unique()

array([nan, 'dicht besiedelt', 'gering besiedelt',
       'mittlere Besiedlungsdichte'], dtype=object)

In [200]:
comp_inflation_party_pivot_zensus.Besiedelung=comp_inflation_party_pivot_zensus.Besiedelung.str.replace('dicht besiedelt',"Grossstadt")
comp_inflation_party_pivot_zensus.Besiedelung=comp_inflation_party_pivot_zensus.Besiedelung.str.replace('gering besiedelt',"Dorf")
comp_inflation_party_pivot_zensus.Besiedelung=comp_inflation_party_pivot_zensus.Besiedelung.str.replace('mittlere Besiedlungsdichte',"Kleinstadt")

In [201]:
comp_inflation_party_pivot_zensus.Geschlecht.unique()

array([nan, 'Männlich', 'Weiblich'], dtype=object)

In [202]:
comp_inflation_party_pivot_zensus.Geschlecht=comp_inflation_party_pivot_zensus.Geschlecht.str.replace("Männlich","Maennlich")

In [203]:
perc_share_df=comp_inflation_party_pivot_zensus[comp_inflation_party_pivot_zensus.columns[~comp_inflation_party_pivot_zensus.columns.str.contains("absolute")]]

In [204]:
perc_share_df

,Alter,Besiedelung,Geschlecht,OstWest,AM DRITTWICHTIGSTEN_percent_2021,AM VIERTWICHTIGSTEN_percent_2021,AM WICHTIGSTEN_percent_2021,AM ZWEITWICHTIGSTEN_percent_2021,AM DRITTWICHTIGSTEN_percent_2023,AM VIERTWICHTIGSTEN_percent_2023,...,WUERDE NICHT WAEHLEN_percent_2021,AFD_percent_2023,ANDERE PARTEI_percent_2023,CDU-CSU_percent_2023,DIE GRUENEN_percent_2023,DIE LINKE_percent_2023,FDP_percent_2023,SPD_percent_2023,WUERDE NICHT WAEHLEN_percent_2023,Share
0,NaN,NaN,NaN,Berlin,25.477707,47.770701,7.006369,19.745223,29.515419,35.682819,...,7.386364,9.793814,4.123711,15.463918,23.195876,20.103093,4.639175,18.041237,4.639175,3.989901
1,NaN,NaN,NaN,Ost,25.974026,51.048951,6.293706,16.683317,26.083815,40.895954,...,6.560135,21.171171,4.324324,20.900901,10.540541,11.621622,5.945946,19.369369,6.126126,16.433332
2,NaN,NaN,NaN,West,24.885496,56.284987,4.580153,14.249364,28.441907,39.355272,...,4.331683,8.271605,4.074074,27.572016,22.469136,3.127572,9.465021,20.082305,4.938272,79.519741
3,NaN,NaN,Maennlich,NaN,24.984147,55.231452,5.009512,14.774889,29.456328,41.844920,...,4.730417,14.524556,4.231975,25.496343,16.614420,5.799373,9.247649,18.808777,5.276907,48.323387
4,NaN,NaN,Maennlich,Berlin,19.718310,57.746479,5.633803,16.901408,33.962264,42.452830,...,7.692308,11.111111,5.555556,16.666667,21.111111,18.888889,4.444444,17.777778,4.444444,1.910200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,65plus,NaN,Maennlich,NaN,24.481328,57.883817,2.489627,15.145228,28.251121,46.935725,...,3.987241,10.033445,1.170569,34.280936,10.367893,6.020067,6.856187,27.090301,4.180602,12.463294
59,65plus,NaN,Weiblich,NaN,24.764151,57.311321,2.122642,15.801887,25.829726,44.877345,...,4.638219,6.560284,2.482270,30.851064,14.539007,5.496454,6.205674,29.078014,4.787234,15.997430
60,65plus,Grossstadt,NaN,NaN,26.351351,52.702703,4.054054,16.891892,25.609756,50.813008,...,5.617978,7.476636,1.869159,23.364486,10.747664,7.943925,7.943925,36.448598,4.205607,9.247415
61,65plus,Dorf,NaN,NaN,24.166667,60.833333,1.666667,13.333333,26.984127,44.179894,...,6.012658,7.619048,2.539683,39.365079,13.015873,3.174603,6.031746,24.444444,3.809524,6.025729


In [205]:
perc_share_df.to_csv("perc_share_df.csv")

In [206]:
perc_share_df

,Alter,Besiedelung,Geschlecht,OstWest,AM DRITTWICHTIGSTEN_percent_2021,AM VIERTWICHTIGSTEN_percent_2021,AM WICHTIGSTEN_percent_2021,AM ZWEITWICHTIGSTEN_percent_2021,AM DRITTWICHTIGSTEN_percent_2023,AM VIERTWICHTIGSTEN_percent_2023,...,WUERDE NICHT WAEHLEN_percent_2021,AFD_percent_2023,ANDERE PARTEI_percent_2023,CDU-CSU_percent_2023,DIE GRUENEN_percent_2023,DIE LINKE_percent_2023,FDP_percent_2023,SPD_percent_2023,WUERDE NICHT WAEHLEN_percent_2023,Share
0,NaN,NaN,NaN,Berlin,25.477707,47.770701,7.006369,19.745223,29.515419,35.682819,...,7.386364,9.793814,4.123711,15.463918,23.195876,20.103093,4.639175,18.041237,4.639175,3.989901
1,NaN,NaN,NaN,Ost,25.974026,51.048951,6.293706,16.683317,26.083815,40.895954,...,6.560135,21.171171,4.324324,20.900901,10.540541,11.621622,5.945946,19.369369,6.126126,16.433332
2,NaN,NaN,NaN,West,24.885496,56.284987,4.580153,14.249364,28.441907,39.355272,...,4.331683,8.271605,4.074074,27.572016,22.469136,3.127572,9.465021,20.082305,4.938272,79.519741
3,NaN,NaN,Maennlich,NaN,24.984147,55.231452,5.009512,14.774889,29.456328,41.844920,...,4.730417,14.524556,4.231975,25.496343,16.614420,5.799373,9.247649,18.808777,5.276907,48.323387
4,NaN,NaN,Maennlich,Berlin,19.718310,57.746479,5.633803,16.901408,33.962264,42.452830,...,7.692308,11.111111,5.555556,16.666667,21.111111,18.888889,4.444444,17.777778,4.444444,1.910200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,65plus,NaN,Maennlich,NaN,24.481328,57.883817,2.489627,15.145228,28.251121,46.935725,...,3.987241,10.033445,1.170569,34.280936,10.367893,6.020067,6.856187,27.090301,4.180602,12.463294
59,65plus,NaN,Weiblich,NaN,24.764151,57.311321,2.122642,15.801887,25.829726,44.877345,...,4.638219,6.560284,2.482270,30.851064,14.539007,5.496454,6.205674,29.078014,4.787234,15.997430
60,65plus,Grossstadt,NaN,NaN,26.351351,52.702703,4.054054,16.891892,25.609756,50.813008,...,5.617978,7.476636,1.869159,23.364486,10.747664,7.943925,7.943925,36.448598,4.205607,9.247415
61,65plus,Dorf,NaN,NaN,24.166667,60.833333,1.666667,13.333333,26.984127,44.179894,...,6.012658,7.619048,2.539683,39.365079,13.015873,3.174603,6.031746,24.444444,3.809524,6.025729


In [207]:
perc_share_df_no_berlin=perc_share_df[perc_share_df["OstWest"]!="Berlin"]
perc_share_df_no_berlin

,Alter,Besiedelung,Geschlecht,OstWest,AM DRITTWICHTIGSTEN_percent_2021,AM VIERTWICHTIGSTEN_percent_2021,AM WICHTIGSTEN_percent_2021,AM ZWEITWICHTIGSTEN_percent_2021,AM DRITTWICHTIGSTEN_percent_2023,AM VIERTWICHTIGSTEN_percent_2023,...,WUERDE NICHT WAEHLEN_percent_2021,AFD_percent_2023,ANDERE PARTEI_percent_2023,CDU-CSU_percent_2023,DIE GRUENEN_percent_2023,DIE LINKE_percent_2023,FDP_percent_2023,SPD_percent_2023,WUERDE NICHT WAEHLEN_percent_2023,Share
1,NaN,NaN,NaN,Ost,25.974026,51.048951,6.293706,16.683317,26.083815,40.895954,...,6.560135,21.171171,4.324324,20.900901,10.540541,11.621622,5.945946,19.369369,6.126126,16.433332
2,NaN,NaN,NaN,West,24.885496,56.284987,4.580153,14.249364,28.441907,39.355272,...,4.331683,8.271605,4.074074,27.572016,22.469136,3.127572,9.465021,20.082305,4.938272,79.519741
3,NaN,NaN,Maennlich,NaN,24.984147,55.231452,5.009512,14.774889,29.456328,41.844920,...,4.730417,14.524556,4.231975,25.496343,16.614420,5.799373,9.247649,18.808777,5.276907,48.323387
5,NaN,NaN,Maennlich,Ost,27.983539,50.411523,5.761317,15.843621,25.914634,44.512195,...,7.119741,25.088968,3.914591,21.708185,9.964413,10.498221,5.338078,16.192171,7.295374,7.948821
6,NaN,NaN,Maennlich,West,23.921569,57.352941,4.607843,14.117647,30.701754,40.620783,...,3.341289,10.063391,4.278922,27.812995,19.255151,2.773376,11.331220,20.047544,4.437401,38.464365
7,NaN,NaN,Weiblich,NaN,25.601040,53.086420,5.458090,15.854451,26.030928,37.714777,...,5.687465,9.739900,3.984505,24.571112,21.361372,7.194245,7.028224,20.863309,5.257333,51.619587
9,NaN,NaN,Weiblich,Ost,24.218750,51.562500,6.640625,17.578125,26.002766,37.759336,...,6.007067,17.247706,4.770642,20.183486,10.825688,12.844037,6.605505,22.568807,4.954128,8.484510
10,NaN,NaN,Weiblich,West,25.902335,55.095541,4.564756,14.437367,26.060606,38.316498,...,5.426357,6.298533,3.710095,27.523727,25.970664,3.364970,7.420190,20.276100,5.435720,41.055375
11,NaN,Grossstadt,NaN,NaN,27.411168,53.130288,5.922166,13.536379,28.964059,38.477801,...,4.709141,9.299363,3.184713,16.560510,25.987261,10.318471,6.878981,22.675159,5.095541,34.773857
13,NaN,Grossstadt,NaN,Ost,34.027778,51.388889,6.250000,8.333333,24.714829,37.262357,...,4.444444,16.826923,4.807692,17.307692,19.230769,12.980769,4.807692,19.230769,4.807692,3.841123


In [208]:
perc_share_df_no_berlin[['AM DRITTWICHTIGSTEN_percent_2021', 'AM VIERTWICHTIGSTEN_percent_2021',
       'AM WICHTIGSTEN_percent_2021', 'AM ZWEITWICHTIGSTEN_percent_2021',
       'AM DRITTWICHTIGSTEN_percent_2023', 'AM VIERTWICHTIGSTEN_percent_2023',
       'AM WICHTIGSTEN_percent_2023', 'AM ZWEITWICHTIGSTEN_percent_2023',
       'AFD_percent_2021', 'ANDERE PARTEI_percent_2021',
       'CDU-CSU_percent_2021', 'DIE GRUENEN_percent_2021',
       'DIE LINKE_percent_2021', 'FDP_percent_2021', 'SPD_percent_2021',
       'WUERDE NICHT WAEHLEN_percent_2021', 'AFD_percent_2023',
       'ANDERE PARTEI_percent_2023', 'CDU-CSU_percent_2023',
       'DIE GRUENEN_percent_2023', 'DIE LINKE_percent_2023',
       'FDP_percent_2023', 'SPD_percent_2023',
       'WUERDE NICHT WAEHLEN_percent_2023', 'Share']]=perc_share_df_no_berlin[['AM DRITTWICHTIGSTEN_percent_2021', 'AM VIERTWICHTIGSTEN_percent_2021',
       'AM WICHTIGSTEN_percent_2021', 'AM ZWEITWICHTIGSTEN_percent_2021',
       'AM DRITTWICHTIGSTEN_percent_2023', 'AM VIERTWICHTIGSTEN_percent_2023',
       'AM WICHTIGSTEN_percent_2023', 'AM ZWEITWICHTIGSTEN_percent_2023',
       'AFD_percent_2021', 'ANDERE PARTEI_percent_2021',
       'CDU-CSU_percent_2021', 'DIE GRUENEN_percent_2021',
       'DIE LINKE_percent_2021', 'FDP_percent_2021', 'SPD_percent_2021',
       'WUERDE NICHT WAEHLEN_percent_2021', 'AFD_percent_2023',
       'ANDERE PARTEI_percent_2023', 'CDU-CSU_percent_2023',
       'DIE GRUENEN_percent_2023', 'DIE LINKE_percent_2023',
       'FDP_percent_2023', 'SPD_percent_2023',
       'WUERDE NICHT WAEHLEN_percent_2023', 'Share']].round(1)
perc_share_df_no_berlin

/var/folders/jf/2jm804y50jv2nrhclzdyvvdh0000gn/T/ipykernel_11116/3728253973.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  perc_share_df_no_berlin[['AM DRITTWICHTIGSTEN_percent_2021', 'AM VIERTWICHTIGSTEN_percent_2021',


,Alter,Besiedelung,Geschlecht,OstWest,AM DRITTWICHTIGSTEN_percent_2021,AM VIERTWICHTIGSTEN_percent_2021,AM WICHTIGSTEN_percent_2021,AM ZWEITWICHTIGSTEN_percent_2021,AM DRITTWICHTIGSTEN_percent_2023,AM VIERTWICHTIGSTEN_percent_2023,...,WUERDE NICHT WAEHLEN_percent_2021,AFD_percent_2023,ANDERE PARTEI_percent_2023,CDU-CSU_percent_2023,DIE GRUENEN_percent_2023,DIE LINKE_percent_2023,FDP_percent_2023,SPD_percent_2023,WUERDE NICHT WAEHLEN_percent_2023,Share
1,NaN,NaN,NaN,Ost,26.0,51.0,6.3,16.7,26.1,40.9,...,6.6,21.2,4.3,20.9,10.5,11.6,5.9,19.4,6.1,16.4
2,NaN,NaN,NaN,West,24.9,56.3,4.6,14.2,28.4,39.4,...,4.3,8.3,4.1,27.6,22.5,3.1,9.5,20.1,4.9,79.5
3,NaN,NaN,Maennlich,NaN,25.0,55.2,5.0,14.8,29.5,41.8,...,4.7,14.5,4.2,25.5,16.6,5.8,9.2,18.8,5.3,48.3
5,NaN,NaN,Maennlich,Ost,28.0,50.4,5.8,15.8,25.9,44.5,...,7.1,25.1,3.9,21.7,10.0,10.5,5.3,16.2,7.3,7.9
6,NaN,NaN,Maennlich,West,23.9,57.4,4.6,14.1,30.7,40.6,...,3.3,10.1,4.3,27.8,19.3,2.8,11.3,20.0,4.4,38.5
7,NaN,NaN,Weiblich,NaN,25.6,53.1,5.5,15.9,26.0,37.7,...,5.7,9.7,4.0,24.6,21.4,7.2,7.0,20.9,5.3,51.6
9,NaN,NaN,Weiblich,Ost,24.2,51.6,6.6,17.6,26.0,37.8,...,6.0,17.2,4.8,20.2,10.8,12.8,6.6,22.6,5.0,8.5
10,NaN,NaN,Weiblich,West,25.9,55.1,4.6,14.4,26.1,38.3,...,5.4,6.3,3.7,27.5,26.0,3.4,7.4,20.3,5.4,41.1
11,NaN,Grossstadt,NaN,NaN,27.4,53.1,5.9,13.5,29.0,38.5,...,4.7,9.3,3.2,16.6,26.0,10.3,6.9,22.7,5.1,34.8
13,NaN,Grossstadt,NaN,Ost,34.0,51.4,6.2,8.3,24.7,37.3,...,4.4,16.8,4.8,17.3,19.2,13.0,4.8,19.2,4.8,3.8


In [221]:
perc_share_df_no_berlin[perc_share_df_no_berlin["Alter"]=="U30"].iloc[3]

Alter                                      U30
Besiedelung                                NaN
Geschlecht                           Maennlich
OstWest                                    NaN
AM DRITTWICHTIGSTEN_percent_2021          26.9
AM VIERTWICHTIGSTEN_percent_2021          52.0
AM WICHTIGSTEN_percent_2021                7.4
AM ZWEITWICHTIGSTEN_percent_2021          13.7
AM DRITTWICHTIGSTEN_percent_2023          32.3
AM VIERTWICHTIGSTEN_percent_2023          26.8
AM WICHTIGSTEN_percent_2023               19.9
AM ZWEITWICHTIGSTEN_percent_2023          21.0
AFD_percent_2021                          11.2
ANDERE PARTEI_percent_2021                 7.7
CDU-CSU_percent_2021                       9.0
DIE GRUENEN_percent_2021                  27.5
DIE LINKE_percent_2021                    10.3
FDP_percent_2021                          18.5
SPD_percent_2021                          10.7
WUERDE NICHT WAEHLEN_percent_2021          5.2
AFD_percent_2023                          10.6
ANDERE PARTEI

In [220]:
perc_share_df_no_berlin[perc_share_df_no_berlin["Alter"]=="U30"].iloc[4]

Alter                                     U30
Besiedelung                               NaN
Geschlecht                           Weiblich
OstWest                                   NaN
AM DRITTWICHTIGSTEN_percent_2021         30.3
AM VIERTWICHTIGSTEN_percent_2021         42.7
AM WICHTIGSTEN_percent_2021              10.7
AM ZWEITWICHTIGSTEN_percent_2021         16.3
AM DRITTWICHTIGSTEN_percent_2023         25.8
AM VIERTWICHTIGSTEN_percent_2023         24.4
AM WICHTIGSTEN_percent_2023              22.9
AM ZWEITWICHTIGSTEN_percent_2023         26.9
AFD_percent_2021                          2.6
ANDERE PARTEI_percent_2021                7.9
CDU-CSU_percent_2021                     16.4
DIE GRUENEN_percent_2021                 36.0
DIE LINKE_percent_2021                   10.1
FDP_percent_2021                         10.6
SPD_percent_2021                          9.5
WUERDE NICHT WAEHLEN_percent_2021         6.9
AFD_percent_2023                          7.1
ANDERE PARTEI_percent_2023        

In [209]:
inflation_cols=perc_share_df_no_berlin.columns[perc_share_df_no_berlin.columns.str.contains("percent")][:8]
inflation_cols

Index(['AM DRITTWICHTIGSTEN_percent_2021', 'AM VIERTWICHTIGSTEN_percent_2021',
       'AM WICHTIGSTEN_percent_2021', 'AM ZWEITWICHTIGSTEN_percent_2021',
       'AM DRITTWICHTIGSTEN_percent_2023', 'AM VIERTWICHTIGSTEN_percent_2023',
       'AM WICHTIGSTEN_percent_2023', 'AM ZWEITWICHTIGSTEN_percent_2023'],
      dtype='object')

In [210]:
party_cols=perc_share_df_no_berlin.columns[perc_share_df_no_berlin.columns.str.contains("percent")][8:]
party_cols

Index(['AFD_percent_2021', 'ANDERE PARTEI_percent_2021',
       'CDU-CSU_percent_2021', 'DIE GRUENEN_percent_2021',
       'DIE LINKE_percent_2021', 'FDP_percent_2021', 'SPD_percent_2021',
       'WUERDE NICHT WAEHLEN_percent_2021', 'AFD_percent_2023',
       'ANDERE PARTEI_percent_2023', 'CDU-CSU_percent_2023',
       'DIE GRUENEN_percent_2023', 'DIE LINKE_percent_2023',
       'FDP_percent_2023', 'SPD_percent_2023',
       'WUERDE NICHT WAEHLEN_percent_2023'],
      dtype='object')

In [211]:
json_array=[]
for index, row in perc_share_df_no_berlin.iterrows():
    base_dict=(row[row.index[[0,1,2,3,-1]]].to_dict())
    party_dict=(row[party_cols].to_dict())
    inflation_dict=(row[inflation_cols].to_dict())
    base_dict["wahl"]=party_dict
    base_dict["inflation"]=inflation_dict
    json_array.append(base_dict)
json_array


[{'Alter': nan,
  'Besiedelung': nan,
  'Geschlecht': nan,
  'OstWest': 'Ost',
  'Share': 16.4,
  'wahl': {'AFD_percent_2021': 11.7,
   'ANDERE PARTEI_percent_2021': 5.3,
   'CDU-CSU_percent_2021': 23.8,
   'DIE GRUENEN_percent_2021': 14.3,
   'DIE LINKE_percent_2021': 13.2,
   'FDP_percent_2021': 11.5,
   'SPD_percent_2021': 13.6,
   'WUERDE NICHT WAEHLEN_percent_2021': 6.6,
   'AFD_percent_2023': 21.2,
   'ANDERE PARTEI_percent_2023': 4.3,
   'CDU-CSU_percent_2023': 20.9,
   'DIE GRUENEN_percent_2023': 10.5,
   'DIE LINKE_percent_2023': 11.6,
   'FDP_percent_2023': 5.9,
   'SPD_percent_2023': 19.4,
   'WUERDE NICHT WAEHLEN_percent_2023': 6.1},
  'inflation': {'AM DRITTWICHTIGSTEN_percent_2021': 26.0,
   'AM VIERTWICHTIGSTEN_percent_2021': 51.0,
   'AM WICHTIGSTEN_percent_2021': 6.3,
   'AM ZWEITWICHTIGSTEN_percent_2021': 16.7,
   'AM DRITTWICHTIGSTEN_percent_2023': 26.1,
   'AM VIERTWICHTIGSTEN_percent_2023': 40.9,
   'AM WICHTIGSTEN_percent_2023': 11.6,
   'AM ZWEITWICHTIGSTEN_perce

In [212]:
json_array

[{'Alter': nan,
  'Besiedelung': nan,
  'Geschlecht': nan,
  'OstWest': 'Ost',
  'Share': 16.4,
  'wahl': {'AFD_percent_2021': 11.7,
   'ANDERE PARTEI_percent_2021': 5.3,
   'CDU-CSU_percent_2021': 23.8,
   'DIE GRUENEN_percent_2021': 14.3,
   'DIE LINKE_percent_2021': 13.2,
   'FDP_percent_2021': 11.5,
   'SPD_percent_2021': 13.6,
   'WUERDE NICHT WAEHLEN_percent_2021': 6.6,
   'AFD_percent_2023': 21.2,
   'ANDERE PARTEI_percent_2023': 4.3,
   'CDU-CSU_percent_2023': 20.9,
   'DIE GRUENEN_percent_2023': 10.5,
   'DIE LINKE_percent_2023': 11.6,
   'FDP_percent_2023': 5.9,
   'SPD_percent_2023': 19.4,
   'WUERDE NICHT WAEHLEN_percent_2023': 6.1},
  'inflation': {'AM DRITTWICHTIGSTEN_percent_2021': 26.0,
   'AM VIERTWICHTIGSTEN_percent_2021': 51.0,
   'AM WICHTIGSTEN_percent_2021': 6.3,
   'AM ZWEITWICHTIGSTEN_percent_2021': 16.7,
   'AM DRITTWICHTIGSTEN_percent_2023': 26.1,
   'AM VIERTWICHTIGSTEN_percent_2023': 40.9,
   'AM WICHTIGSTEN_percent_2023': 11.6,
   'AM ZWEITWICHTIGSTEN_perce

In [213]:

import json

with open('data_for_viz.json', 'w', encoding='utf-8') as f:
    json.dump(json_array, f)

In [91]:
cols2021=comp_inflation_party_pivot_zensus.columns[comp_inflation_party_pivot_zensus.columns.str.contains("_percent_2021")]
cols2021

Index(['AM DRITTWICHTIGSTEN_percent_2021', 'AM VIERTWICHTIGSTEN_percent_2021',
       'AM WICHTIGSTEN_percent_2021', 'AM ZWEITWICHTIGSTEN_percent_2021',
       'AFD_percent_2021', 'ANDERE PARTEI_percent_2021',
       'CDU-CSU_percent_2021', 'DIE GRUENEN_percent_2021',
       'DIE LINKE_percent_2021', 'FDP_percent_2021', 'SPD_percent_2021',
       'WUERDE NICHT WAEHLEN_percent_2021'],
      dtype='object')

In [92]:
dictionary={}
for columnname21 in cols2021:
    columnname23=columnname21.replace("2021","2023")
    df21=comp_inflation_party_pivot_zensus[columnname21]
    df23=comp_inflation_party_pivot_zensus[columnname23]
    diff=df23-df21
    dictionary[columnname21.replace("_2021","")]=diff
dictionary

{'AM DRITTWICHTIGSTEN_percent': 0      4.037711
 1      0.109789
 2      3.556411
 3      4.472181
 4     14.243954
         ...    
 58     3.769793
 59     1.065575
 60    -0.741595
 61     2.817460
 62     2.960459
 Length: 63, dtype: float64,
 'AM VIERTWICHTIGSTEN_percent': 0    -12.087881
 1    -10.152997
 2    -16.929715
 3    -13.386532
 4    -15.293649
         ...    
 58   -10.948092
 59   -12.433976
 60    -1.889695
 61   -16.653439
 62   -11.634622
 Length: 63, dtype: float64,
 'AM WICHTIGSTEN_percent': 0     4.006846
 1     5.266987
 2     6.937645
 3     4.838973
 4     2.856763
         ...   
 58    3.339970
 59    5.669566
 60    0.417491
 61    6.269841
 62    4.803315
 Length: 63, dtype: float64,
 'AM ZWEITWICHTIGSTEN_percent': 0     4.043323
 1     4.776221
 2     6.435660
 3     4.075378
 4    -1.807069
         ...   
 58    3.838329
 59    5.698835
 60    2.213799
 61    7.566138
 62    3.870847
 Length: 63, dtype: float64,
 'AFD_percent': 0     4.111996
 1     9

In [93]:
diffs=pd.DataFrame(dictionary)

In [94]:
base_data=comp_inflation_party_pivot_zensus[["Alter",	"Besiedelung",	"Geschlecht",	"OstWest", "Share"]]
base_data

,Alter,Besiedelung,Geschlecht,OstWest,Share
0,NaN,NaN,NaN,Berlin,3.989901
1,NaN,NaN,NaN,Ost,16.433332
2,NaN,NaN,NaN,West,79.519741
3,NaN,NaN,Männlich,NaN,48.323387
4,NaN,NaN,Männlich,Berlin,1.910200
...,...,...,...,...,...
58,Ü65,NaN,Männlich,NaN,12.463294
59,Ü65,NaN,Weiblich,NaN,15.997430
60,Ü65,dicht besiedelt,NaN,NaN,9.247415
61,Ü65,gering besiedelt,NaN,NaN,6.025729


In [103]:
base_diffs=pd.concat([base_data,diffs],axis=1)
base_diffs

,Alter,Besiedelung,Geschlecht,OstWest,Share,AM DRITTWICHTIGSTEN_percent,AM VIERTWICHTIGSTEN_percent,AM WICHTIGSTEN_percent,AM ZWEITWICHTIGSTEN_percent,AFD_percent,ANDERE PARTEI_percent,CDU-CSU_percent,DIE GRUENEN_percent,DIE LINKE_percent,FDP_percent,SPD_percent,WUERDE NICHT WAEHLEN_percent
0,NaN,NaN,NaN,Berlin,3.989901,4.037711,-12.087881,4.006846,4.043323,4.111996,0.146439,-3.854264,-5.213215,10.444002,-0.474461,-2.413308,-2.747188
1,NaN,NaN,NaN,Ost,16.433332,0.109789,-10.152997,5.266987,4.776221,9.480675,-0.974246,-2.900613,-3.757189,-1.582752,-5.576342,5.744474,-0.434008
2,NaN,NaN,NaN,West,79.519741,3.556411,-16.929715,6.937645,6.435660,3.692397,0.526219,-0.109502,-4.841095,-1.740415,-2.993725,4.859532,0.606588
3,NaN,NaN,Männlich,NaN,48.323387,4.472181,-13.386532,4.838973,4.075378,5.216316,0.315393,-0.088601,-3.731460,-2.186385,-3.875444,3.803691,0.546490
4,NaN,NaN,Männlich,Berlin,1.910200,14.243954,-15.293649,2.856763,-1.807069,2.319902,2.258852,1.282051,-5.262515,10.097680,-3.247863,-4.200244,-3.247863
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,Ü65,NaN,Männlich,NaN,12.463294,3.769793,-10.948092,3.339970,3.838329,1.580494,-0.105349,2.383010,-2.869747,-1.794925,-4.786556,5.399711,0.193361
59,Ü65,NaN,Weiblich,NaN,15.997430,1.065575,-12.433976,5.669566,5.698835,4.333939,0.255924,-6.069159,-0.674351,-3.037869,-3.812879,8.855380,0.149015
60,Ü65,dicht besiedelt,NaN,NaN,9.247415,-0.741595,-1.889695,0.417491,2.213799,0.735062,0.183766,-7.534390,-4.420876,2.325948,-0.483041,10.605902,-1.412370
61,Ü65,gering besiedelt,NaN,NaN,6.025729,2.817460,-16.653439,6.269841,7.566138,1.289934,0.640948,5.820776,-0.275266,-4.736789,-4.727748,4.191280,-2.203134


In [104]:
base_diffs.sort_values(by=["AM WICHTIGSTEN_percent"],ascending=False).head(20)

,Alter,Besiedelung,Geschlecht,OstWest,Share,AM DRITTWICHTIGSTEN_percent,AM VIERTWICHTIGSTEN_percent,AM WICHTIGSTEN_percent,AM ZWEITWICHTIGSTEN_percent,AFD_percent,ANDERE PARTEI_percent,CDU-CSU_percent,DIE GRUENEN_percent,DIE LINKE_percent,FDP_percent,SPD_percent,WUERDE NICHT WAEHLEN_percent
51,U30,dicht besiedelt,NaN,NaN,6.608907,4.398148,-32.638889,15.914352,12.326389,2.211357,-1.175204,0.901588,-10.105858,1.973625,-2.852786,8.177088,0.870189
48,U30,NaN,NaN,West,13.635077,-0.825749,-23.034718,14.524002,9.336465,0.638891,-2.100742,3.163287,-1.953680,-3.463254,0.067200,3.850874,-0.202575
49,U30,NaN,Männlich,NaN,8.301270,5.445263,-25.195876,12.502700,7.247914,-0.565578,-0.521932,7.512548,-3.738998,-1.825853,-3.200698,3.253437,-0.912926
53,U30,mittlere Besiedlungsdichte,NaN,NaN,6.699213,-4.294103,-17.317103,12.282848,9.328358,5.719539,-0.361023,2.249708,-3.389604,0.558248,-6.528497,3.285977,-1.534347
50,U30,NaN,Weiblich,NaN,7.959077,-4.530627,-18.323869,12.264911,10.589586,4.429969,-4.162923,-3.194569,-2.016572,3.154637,-3.978237,8.400719,-2.633024
45,U30,NaN,NaN,NaN,16.260346,0.626856,-21.876256,12.215121,9.034279,1.484051,-2.301141,2.468012,-2.582049,0.847954,-3.891382,5.483716,-1.509160
32,30-49,NaN,Weiblich,NaN,13.543008,-1.193473,-17.855478,11.379953,7.668998,6.591128,-1.374156,-1.931855,-7.174542,2.501607,-2.891353,4.619897,-0.340726
35,30-49,mittlere Besiedlungsdichte,NaN,NaN,11.532673,0.725018,-15.953031,8.987426,6.240587,7.195106,-0.412629,0.948064,-9.700353,-1.258429,-3.202787,4.977448,1.453579
46,U30,NaN,NaN,Berlin,0.662670,11.607143,-23.214286,8.928571,2.678571,12.500000,8.333333,-15.000000,-10.000000,28.333333,-5.833333,-3.333333,-15.000000
10,NaN,NaN,Weiblich,West,41.055375,0.158271,-16.779043,8.701237,7.919535,4.145217,0.006391,-0.210984,-5.209353,-0.683264,-3.346391,5.289020,0.009364


In [105]:
abs_numbers=comp_inflation_party_pivot_zensus[comp_inflation_party_pivot_zensus.columns[comp_inflation_party_pivot_zensus.columns.str.contains("absolute")]]

In [107]:
base_abs=pd.concat([base_data,abs_numbers],axis=1)
base_abs

,Alter,Besiedelung,Geschlecht,OstWest,Share,AM DRITTWICHTIGSTEN_absolute_2021,AM VIERTWICHTIGSTEN_absolute_2021,AM WICHTIGSTEN_absolute_2021,AM ZWEITWICHTIGSTEN_absolute_2021,AM DRITTWICHTIGSTEN_absolute_2023,...,SPD_absolute_2021,WUERDE NICHT WAEHLEN_absolute_2021,AFD_absolute_2023,ANDERE PARTEI_absolute_2023,CDU-CSU_absolute_2023,DIE GRUENEN_absolute_2023,DIE LINKE_absolute_2023,FDP_absolute_2023,SPD_absolute_2023,WUERDE NICHT WAEHLEN_absolute_2023
0,NaN,NaN,NaN,Berlin,3.989901,40,75,11,31,67,...,36,13,19,8,30,45,39,9,35,9
1,NaN,NaN,NaN,Ost,16.433332,260,511,63,167,361,...,162,78,235,48,232,117,129,66,215,68
2,NaN,NaN,NaN,West,79.519741,489,1106,90,280,847,...,369,105,201,99,670,546,76,230,488,120
3,NaN,NaN,Männlich,NaN,48.323387,394,871,79,233,661,...,295,93,278,81,488,318,111,177,360,101
4,NaN,NaN,Männlich,Berlin,1.910200,14,41,4,12,36,...,20,7,10,5,15,19,17,4,16,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,Ü65,NaN,Männlich,NaN,12.463294,118,279,12,73,189,...,136,25,60,7,205,62,36,41,162,25
59,Ü65,NaN,Weiblich,NaN,15.997430,105,243,9,67,179,...,109,25,37,14,174,82,31,35,164,27
60,Ü65,dicht besiedelt,NaN,NaN,9.247415,39,78,6,25,63,...,46,10,16,4,50,23,17,17,78,9
61,Ü65,gering besiedelt,NaN,NaN,6.025729,58,146,4,32,102,...,64,19,24,8,124,41,10,19,77,12


In [108]:
base_abs[base_abs.Alter=="U30"]

,Alter,Besiedelung,Geschlecht,OstWest,Share,AM DRITTWICHTIGSTEN_absolute_2021,AM VIERTWICHTIGSTEN_absolute_2021,AM WICHTIGSTEN_absolute_2021,AM ZWEITWICHTIGSTEN_absolute_2021,AM DRITTWICHTIGSTEN_absolute_2023,...,SPD_absolute_2021,WUERDE NICHT WAEHLEN_absolute_2021,AFD_absolute_2023,ANDERE PARTEI_absolute_2023,CDU-CSU_absolute_2023,DIE GRUENEN_absolute_2023,DIE LINKE_absolute_2023,FDP_absolute_2023,SPD_absolute_2023,WUERDE NICHT WAEHLEN_absolute_2023
45,U30,NaN,NaN,NaN,16.260346,101,167,32,53,169,...,43,25,40,25,67,130,50,50,71,20
46,U30,NaN,NaN,Berlin,0.662670,5,6,2,3,12,...,4,3,3,2,0,6,8,1,4,0
47,U30,NaN,NaN,Ost,1.962600,17,32,12,14,35,...,5,6,12,8,15,20,20,4,19,4
48,U30,NaN,NaN,West,13.635077,79,129,18,36,122,...,34,16,25,15,52,104,22,45,48,16
49,U30,NaN,Männlich,NaN,8.301270,47,91,13,24,94,...,25,12,25,17,39,56,20,36,33,10
50,U30,NaN,Weiblich,NaN,7.959077,54,76,19,29,72,...,18,13,15,8,28,72,28,14,38,9
51,U30,dicht besiedelt,NaN,NaN,6.608907,29,60,7,12,60,...,11,6,9,7,18,56,23,11,25,8
52,U30,gering besiedelt,NaN,NaN,2.952226,28,44,12,20,41,...,13,11,10,5,21,27,6,13,16,5
53,U30,mittlere Besiedlungsdichte,NaN,NaN,6.699213,42,60,11,21,66,...,19,8,21,13,28,47,21,26,30,7
